# Transfer Learning

## Loading YAMnet

In [1]:
import os
import glob

from IPython import display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_io as tfio


2024-08-16 10:15:46.355916: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-16 10:15:49.614519: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-16 10:15:51.604143: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-16 10:15:53.073220: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-16 10:15:53.380445: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-16 10:15:55.117947: I tensorflow/core/platform/cpu_feature_gu

In [ ]:
yamnet_model_handle = 'https://tfhub.dev/google/yamnet/1'
yamnet_model = hub.load(yamnet_model_handle)


In [ ]:
def load_wav_16k_mono(filename):
    """ Load a WAV file, convert it to a float tensor, resample to 16 kHz single-channel audio. """
    file_contents = tf.io.read_file(filename)
    wav, sample_rate = tf.audio.decode_wav(
          file_contents,
          desired_channels=1)
    wav = tf.squeeze(wav, axis=-1)
    sample_rate = tf.cast(sample_rate, dtype=tf.int64)
    wav = tfio.audio.resample(wav, rate_in=sample_rate, rate_out=16000)
    return wav

In [ ]:
class_map_path = yamnet_model.class_map_path().numpy().decode('utf-8')
class_names =list(pd.read_csv(class_map_path)['display_name'])

In [ ]:
testing_wav_file_name='/home/focus/Bureau/Audio_Node/src/resources/ray_audio/test_wav/Beep*.wav'
testing_wav_data = load_wav_16k_mono(testing_wav_file_name)

_ = plt.plot(testing_wav_data)

# Play the audio file.
display.Audio(testing_wav_data,rate=16000)

In [59]:
scores, embeddings, spectrogram = yamnet_model(testing_wav_data)
class_scores = tf.reduce_mean(scores, axis=0)
top_class = tf.argmax(class_scores)
inferred_class = class_names[top_class]

print(f'The main sound is: {inferred_class}')
print(f'The embeddings shape: {embeddings.shape}')


The main sound is: Beep, bleep
The embeddings shape: (20, 1024)


In [60]:
embeddings

<tf.Tensor: shape=(20, 1024), dtype=float32, numpy=
array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.21649547, ..., 0.12052732, 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.23321708, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.27158436, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.5796338 , 0.        , ..., 0.        , 0.        ,
        0.        ]], dtype=float32)>

## Starting 

In [ ]:
my_classes = ['Beep', 'Music','Silence','Robot Moving','Speech']
map_class_to_id = {'Beep':0, 'Music':1,'Silence':2,'Robot Moving':3,'Speech':4}


In [ ]:
# Load yamnet CSV file
labels_df = pd.read_csv('/home/focus/Bureau/Audio_Node/src/resources/ray_audio/class_labels_indices.csv')

In [ ]:
yamnet_class_labels = {
    "silence": "/m/07p6fty",      # Example MID for silence
    "speech": "/m/09x0r",      # Example MID for speech
    "beep": "/m/0b7l4",        # Example MID for beep
    "robot_moving": "/m/09c6s", # Example MID for robot moving
    "music": "/m/0p8n5"        # Example MID for music
}

In [24]:
import tensorflow as tf

def print_tfrecord_content(filename):
    # Create a TFRecordDataset from the file
    dataset = tf.data.TFRecordDataset(filename)

    # Iterate over the dataset and print raw records
    for record in dataset.take(5):  # Limit to the first 5 records for inspection
        # Print the raw serialized example
        print("Raw Record:")
        print(record.numpy())

        # Optionally, decode the record
        example = tf.train.SequenceExample()
        example.ParseFromString(record.numpy())
        print("Decoded Example:")
        print(example) 

# Replace with the path to one of your .tfrecord files
print_tfrecord_content('/home/focus/Bureau/Audio_Node/src/resources/ray_audio/features/audioset_v1_embeddings/bal_train/ZY.tfrecord')



Raw Record:
b'\nm\n\x1e\n\x12start_time_seconds\x12\x08\x12\x06\n\x04\x00\x00\x96C\n\x10\n\x06labels\x12\x06\x1a\x04\n\x02\x00<\n\x1c\n\x10end_time_seconds\x12\x08\x12\x06\n\x04\x00\x00\x9bC\n\x1b\n\x08video_id\x12\x0f\n\r\n\x0bZYjtrPCAlTU\x12\xf1\n\n\xee\n\n\x0faudio_embedding\x12\xda\n\n\x86\x01\n\x83\x01\n\x80\x01\x8d\x80F^\x9c[\x14]\xa1\xdd\xabz.\x05\x8c\x00hx\xa4\xb9\xd9.\xff\x06Z\x8fH\x8d\x92\xa7\x00\x00\xf5Z\xa4\xad\x00Q\x00z\x90_\xc9DBq0\xb0\xbfR\x01\xfft\x12\xbd\x8d5}\x9eP\xff\xc7\x00D\x00lt\xff\x00\xa7\xa0\xff\x93\x00y\x80\xafc\xa8b~\x00I%\xb1f\x00\xab\xa9\xbc\xd4\xc4w\xab\xd0\xf09\xff\xffw!-\x00\xff\x9f\x9c\x00\x8de\xb1\xa9\x9b<\xbcG\xdd\x00)\x00\x00\x1b\x8d\x91\xba\xd3\xdd\x00\xff\n\x86\x01\n\x83\x01\n\x80\x01\x99P[r\xa8%\\Y\xa8\xe3\x88H%5v\x1e\x8dn\xd6\xc5\x93>\xc2\x12\xab|[\x9d\x82\xf0Be\xd0u\x93\xb6;=\x1e\x860\x8f\xa5?\xb6cf\x83`\xafc\xffQN\xff\xa7\xb7ICg\x8c\xff\x19d\x00\xba\xc3\x8c\x82\xa4\xb6\xafp\x1dc\x91\xc2\x93\xbb$~\x88\x81(;\x99\x00t\x93qv\x11\x95o\x82}E\x8b\xf5\

In [82]:
!grep Beep class_labels_indices.csv

481,/m/02fs_r,"Beep, bleep"


In [83]:
class_label_index = !grep Beep class_labels_indices.csv

In [86]:
print (class_label_index[0].split(",")[1])

/m/02fs_r


In [89]:
!grep /m/02fs_r balanced_train_segments.csv |head

0gi_2J5wEO8, 410.000, 420.000, "/m/02fs_r"
2E_cfP2PLUQ, 40.000, 50.000, "/m/02fs_r"
3cGGjixLbY0, 30.000, 40.000, "/m/02fs_r,/m/09x0r"
7H09aM-_nZc, 0.000, 10.000, "/m/01z47d,/m/02fs_r,/m/04rlf"
9O2Keu6o3i0, 170.000, 180.000, "/m/02fs_r,/m/09x0r,/m/0fx9l,/m/0ytgt"
A_WqrhcgNMM, 280.000, 290.000, "/m/02fs_r,/m/09x0r"
BjKEzvmaQAA, 100.000, 110.000, "/m/02fs_r,/m/04rlf,/m/09x0r"
CDwk_DbprX4, 30.000, 40.000, "/m/02fs_r,/m/09x0r,/m/0c3f7m"
CSfbHfwsIVE, 20.000, 30.000, "/m/01y3hg,/m/02fs_r,/m/07pp_mv,/m/09x0r"
D14QMp6dz0U, 2.000, 12.000, "/m/02fs_r,/m/09x0r"


In [91]:
examples = !grep /m/0l15bq balanced_train_segments.csv | head -2 | cut -c -11
print (examples)

['0FMdORf5iGs', '1IxBagCJeZc']


In [92]:
tfrecord_prefixes = set([i[:2] for i in examples])

In [94]:
tfrecord_filenames = ["/home/focus/Bureau/Audio_Node/src/resources/ray_audio/features/audioset_v1_embeddings/bal_train/" + i + ".tfrecord" for i in tfrecord_prefixes]

In [95]:
/home/focus/Bureau/Audio_Node/src/resources/ray_audio/features/audioset_v1_embeddings/bal_train

['/home/focus/Bureau/Audio_Node/src/resources/ray_audio/features/audioset_v1_embeddings/bal_train/1I.tfrecord',
 '/home/focus/Bureau/Audio_Node/src/resources/ray_audio/features/audioset_v1_embeddings/bal_train/0F.tfrecord']

In [107]:
import csv
import pandas as pd

def parse_csv(csv_file):
    # Initialize lists to store the data
    ytids = []
    start_seconds = []
    end_seconds = []
    positive_labels = []

    # Read the CSV file manually
    with open(csv_file, 'r', newline='', encoding='utf-8') as file:
        reader = csv.reader(file, delimiter=',', quotechar='"')
        # Skip comment lines
        for row in reader:
            if row and not row[0].startswith('#'):
                ytids.append(row[0].strip())
                start_seconds.append(float(row[1].strip()))
                end_seconds.append(float(row[2].strip()))
                # Convert positive_labels to a list, splitting by comma and stripping quotes
                labels = [label.strip() for label in row[3].strip().strip('"').split(',')]
                positive_labels.append(labels)

    # Create a DataFrame
    df = pd.DataFrame({
        'YTID': ytids,
        'start_seconds': start_seconds,
        'end_seconds': end_seconds,
        'positive_labels': positive_labels
    })

    return df

# Example usage
csv_file = '/home/focus/Bureau/Audio_Node/src/resources/ray_audio/balanced_train_segments.csv'
csv_data = parse_csv(csv_file)
print(csv_data.head())


          YTID  start_seconds  end_seconds positive_labels
0  --PJHxphWEs           30.0         40.0      [/m/09x0r]
1  --ZhevVpy1s           50.0         60.0     [/m/012xff]
2  --aE2O5G5WE            0.0         10.0      [/m/03fwl]
3  --aO5cdqSAg           30.0         40.0    [/t/dd00003]
4  --aaILOrkII          200.0        210.0     [/m/032s66]


In [159]:
def load_data_from_tfrecord(tfrecord_filenames):
    # Create a dataset from TFRecord files
    dataset = tf.data.TFRecordDataset(tfrecord_filenames)
    
    # Map the parse_tfrecord function to the dataset
    dataset = dataset.map(parse_tfrecord)
    
    # Optionally batch and shuffle the dataset
    dataset = dataset.batch(1)  # Adjust batch size as needed
    return dataset


In [154]:
tf.data.experimental.enable_debug_mode()

In [141]:
tf.config.run_functions_eagerly(True)


In [32]:
import tensorflow as tf

def extract_audio_embeddings(filename):
    # Create a TFRecordDataset from the file
    dataset = tf.data.TFRecordDataset(filename)

    # Iterate over the dataset and process each record
    for record in dataset:  # Limit to the first 5 records for inspection
        # Parse the raw record into a SequenceExample
        example = tf.train.SequenceExample()
        example.ParseFromString(record.numpy())

        # Access the 'audio_embedding' feature
        audio_embedding_feature = example.feature_lists.feature_list['audio_embedding']

        # Extract the embeddings and print them
        for i, embedding in enumerate(audio_embedding_feature.feature):
            # Each embedding is stored as a byte list, convert it to an array of integers
            embedding_array = list(embedding.bytes_list.value[0])
            print(f"Audio Embedding {i+1}: {embedding_array}")

        print("\n" + "-"*50 + "\n")

# Replace with the path to one of your .tfrecord files
extract_audio_embeddings('/home/focus/Bureau/Audio_Node/src/resources/ray_audio/features/audioset_v1_embeddings/bal_train/ZY.tfrecord')


Audio Embedding 1: [141, 128, 70, 94, 156, 91, 20, 93, 161, 221, 171, 122, 46, 5, 140, 0, 104, 120, 164, 185, 217, 46, 255, 6, 90, 143, 72, 141, 146, 167, 0, 0, 245, 90, 164, 173, 0, 81, 0, 122, 144, 95, 201, 68, 66, 113, 48, 176, 191, 82, 1, 255, 116, 18, 189, 141, 53, 125, 158, 80, 255, 199, 0, 68, 0, 108, 116, 255, 0, 167, 160, 255, 147, 0, 121, 128, 175, 99, 168, 98, 126, 0, 73, 37, 177, 102, 0, 171, 169, 188, 212, 196, 119, 171, 208, 240, 57, 255, 255, 119, 33, 45, 0, 255, 159, 156, 0, 141, 101, 177, 169, 155, 60, 188, 71, 221, 0, 41, 0, 0, 27, 141, 145, 186, 211, 221, 0, 255]
Labels: [0, 60]
Audio Embedding 2: [153, 80, 91, 114, 168, 37, 92, 89, 168, 227, 136, 72, 37, 53, 118, 30, 141, 110, 214, 197, 147, 62, 194, 18, 171, 124, 91, 157, 130, 240, 66, 101, 208, 117, 147, 182, 59, 61, 30, 134, 48, 143, 165, 63, 182, 99, 102, 131, 96, 175, 99, 255, 81, 78, 255, 167, 183, 73, 67, 103, 140, 255, 25, 100, 0, 186, 195, 140, 130, 164, 182, 175, 112, 29, 99, 145, 194, 147, 187, 36, 126, 1

### Map to class

In [33]:
import tensorflow as tf

def extract_audio_embeddings_with_labels(filename):
    # Create a TFRecordDataset from the file
    dataset = tf.data.TFRecordDataset(filename)

    # Iterate over the dataset and process each record
    for record in dataset:
        # Parse the raw record into a SequenceExample
        example = tf.train.SequenceExample()
        example.ParseFromString(record.numpy())

        # Access the 'audio_embedding' feature
        audio_embedding_feature = example.feature_lists.feature_list['audio_embedding']

        # Access the 'labels' feature
        labels_feature = example.context.feature['labels']

        # Extract the labels
        labels = list(labels_feature.int64_list.value)

        # Extract the embeddings and map them to labels
        embeddings_with_labels = []
        for i, embedding in enumerate(audio_embedding_feature.feature):
            # Each embedding is stored as a byte list, convert it to an array of integers
            embedding_array = list(embedding.bytes_list.value[0])

            # Store the embedding with its corresponding labels
            embeddings_with_labels.append((embedding_array, labels))

            print(f"Audio Embedding {i+1}: {embedding_array}")
            print(f"Labels: {labels}")

        print("\n" + "-"*50 + "\n")

# Replace with the path to one of your .tfrecord files
extract_audio_embeddings_with_labels('/home/focus/Bureau/Audio_Node/src/resources/ray_audio/features/audioset_v1_embeddings/bal_train/ZY.tfrecord')


Audio Embedding 1: [141, 128, 70, 94, 156, 91, 20, 93, 161, 221, 171, 122, 46, 5, 140, 0, 104, 120, 164, 185, 217, 46, 255, 6, 90, 143, 72, 141, 146, 167, 0, 0, 245, 90, 164, 173, 0, 81, 0, 122, 144, 95, 201, 68, 66, 113, 48, 176, 191, 82, 1, 255, 116, 18, 189, 141, 53, 125, 158, 80, 255, 199, 0, 68, 0, 108, 116, 255, 0, 167, 160, 255, 147, 0, 121, 128, 175, 99, 168, 98, 126, 0, 73, 37, 177, 102, 0, 171, 169, 188, 212, 196, 119, 171, 208, 240, 57, 255, 255, 119, 33, 45, 0, 255, 159, 156, 0, 141, 101, 177, 169, 155, 60, 188, 71, 221, 0, 41, 0, 0, 27, 141, 145, 186, 211, 221, 0, 255]
Labels: [0, 60]
Audio Embedding 2: [153, 80, 91, 114, 168, 37, 92, 89, 168, 227, 136, 72, 37, 53, 118, 30, 141, 110, 214, 197, 147, 62, 194, 18, 171, 124, 91, 157, 130, 240, 66, 101, 208, 117, 147, 182, 59, 61, 30, 134, 48, 143, 165, 63, 182, 99, 102, 131, 96, 175, 99, 255, 81, 78, 255, 167, 183, 73, 67, 103, 140, 255, 25, 100, 0, 186, 195, 140, 130, 164, 182, 175, 112, 29, 99, 145, 194, 147, 187, 36, 126, 1

2024-08-16 10:08:16.815604: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [2]:

def extract_specific_class_embeddings(filename, target_class):
    # Create a TFRecordDataset from the file
    dataset = tf.data.TFRecordDataset(filename)

    # List to store embeddings related to the target class
    target_class_embeddings = []

    # Iterate over the dataset and process each record
    for record in dataset:
        # Parse the raw record into a SequenceExample
        example = tf.train.SequenceExample()
        example.ParseFromString(record.numpy())

        # Access the 'audio_embedding' feature
        audio_embedding_feature = example.feature_lists.feature_list['audio_embedding']

        # Access the 'labels' feature
        labels_feature = example.context.feature['labels']

        # Extract the labels
        labels = list(labels_feature.int64_list.value)

        # Check if the target class is in the labels
        if target_class in labels:
            # Extract the embeddings associated with the target class
            for i, embedding in enumerate(audio_embedding_feature.feature):
                embedding_array = list(embedding.bytes_list.value[0])
                target_class_embeddings.append(embedding_array)

                # Optionally, print the embeddings for inspection
                print(f"Audio Embedding {i+1} for class {target_class}: {embedding_array}")

    # Return the list of embeddings related to the target class
    return target_class_embeddings

# Replace with the path to one of your .tfrecord files and the desired class ID
target_class_id = 0 # Replace with the integer ID of "/m/012xff"
extracted_embeddings = extract_specific_class_embeddings('/home/focus/Bureau/Audio_Node/src/resources/ray_audio/features/audioset_v1_embeddings/bal_train/ZY.tfrecord', target_class_id)

# Print the number of extracted embeddings for the target class
print(f"Total number of embeddings extracted for class {target_class_id}: {len(extracted_embeddings)}")


Audio Embedding 1 for class 0: [141, 128, 70, 94, 156, 91, 20, 93, 161, 221, 171, 122, 46, 5, 140, 0, 104, 120, 164, 185, 217, 46, 255, 6, 90, 143, 72, 141, 146, 167, 0, 0, 245, 90, 164, 173, 0, 81, 0, 122, 144, 95, 201, 68, 66, 113, 48, 176, 191, 82, 1, 255, 116, 18, 189, 141, 53, 125, 158, 80, 255, 199, 0, 68, 0, 108, 116, 255, 0, 167, 160, 255, 147, 0, 121, 128, 175, 99, 168, 98, 126, 0, 73, 37, 177, 102, 0, 171, 169, 188, 212, 196, 119, 171, 208, 240, 57, 255, 255, 119, 33, 45, 0, 255, 159, 156, 0, 141, 101, 177, 169, 155, 60, 188, 71, 221, 0, 41, 0, 0, 27, 141, 145, 186, 211, 221, 0, 255]
Audio Embedding 2 for class 0: [153, 80, 91, 114, 168, 37, 92, 89, 168, 227, 136, 72, 37, 53, 118, 30, 141, 110, 214, 197, 147, 62, 194, 18, 171, 124, 91, 157, 130, 240, 66, 101, 208, 117, 147, 182, 59, 61, 30, 134, 48, 143, 165, 63, 182, 99, 102, 131, 96, 175, 99, 255, 81, 78, 255, 167, 183, 73, 67, 103, 140, 255, 25, 100, 0, 186, 195, 140, 130, 164, 182, 175, 112, 29, 99, 145, 194, 147, 187, 36

2024-08-16 10:16:23.441035: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [7]:
import tensorflow as tf
import os

def extract_and_save_embeddings_for_class(directory, target_class, max_files=50, output_dir='embeddings'):
    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # List all .tfrecord files in the directory
    tfrecord_files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.tfrecord')]

    total_embeddings = 0

    # Process the first 'max_files' files
    for filename in tfrecord_files[:max_files]:
        print(f"Processing file: {filename}")
        
        # Create a TFRecordDataset from the file
        dataset = tf.data.TFRecordDataset(filename)

        # Iterate over the dataset and process each record
        for record_idx, record in enumerate(dataset):
            # Parse the raw record into a SequenceExample
            example = tf.train.SequenceExample()
            example.ParseFromString(record.numpy())

            # Access the 'audio_embedding' feature
            audio_embedding_feature = example.feature_lists.feature_list['audio_embedding']
            
            # Access the 'labels' feature
            context = example.context.feature
            labels = context['labels'].int64_list.value
            
            # Check if the target class is in the labels
            if int(target_class) in labels:
                print(f"Target class {target_class} found in record {record_idx}.")
                
                # Collect the 10 embeddings for this audio clip
                embeddings = []
                for embedding in audio_embedding_feature.feature:
                    embedding_array = list(embedding.bytes_list.value[0])
                    embeddings.append(embedding_array)
                
                # Save embeddings to a file
                output_file = os.path.join(output_dir, f'embeddings_{filename[-6:]}_{record_idx}.npy')
                np.save(output_file, embeddings)
                print(f"Saved embeddings to {output_file}")

                total_embeddings += len(embeddings)

    print(f"Total embeddings saved for class {target_class}: {total_embeddings}")

# Replace with the target class and the directory containing the .tfrecord files
extract_and_save_embeddings_for_class('/home/focus/Bureau/Audio_Node/src/resources/ray_audio/features/audioset_v1_embeddings/bal_train', '481')


Processing file: /home/focus/Bureau/Audio_Node/src/resources/ray_audio/features/audioset_v1_embeddings/bal_train/r9.tfrecord
Processing file: /home/focus/Bureau/Audio_Node/src/resources/ray_audio/features/audioset_v1_embeddings/bal_train/OP.tfrecord
Processing file: /home/focus/Bureau/Audio_Node/src/resources/ray_audio/features/audioset_v1_embeddings/bal_train/oU.tfrecord
Processing file: /home/focus/Bureau/Audio_Node/src/resources/ray_audio/features/audioset_v1_embeddings/bal_train/eI.tfrecord
Processing file: /home/focus/Bureau/Audio_Node/src/resources/ray_audio/features/audioset_v1_embeddings/bal_train/6n.tfrecord
Processing file: /home/focus/Bureau/Audio_Node/src/resources/ray_audio/features/audioset_v1_embeddings/bal_train/He.tfrecord
Processing file: /home/focus/Bureau/Audio_Node/src/resources/ray_audio/features/audioset_v1_embeddings/bal_train/8x.tfrecord
Processing file: /home/focus/Bureau/Audio_Node/src/resources/ray_audio/features/audioset_v1_embeddings/bal_train/bT.tfrecord


2024-08-16 10:30:22.308760: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Processing file: /home/focus/Bureau/Audio_Node/src/resources/ray_audio/features/audioset_v1_embeddings/bal_train/5n.tfrecord
Processing file: /home/focus/Bureau/Audio_Node/src/resources/ray_audio/features/audioset_v1_embeddings/bal_train/3v.tfrecord
Processing file: /home/focus/Bureau/Audio_Node/src/resources/ray_audio/features/audioset_v1_embeddings/bal_train/A_.tfrecord
Target class 481 found in record 5.
Saved embeddings to embeddings/embeddings_record_5.npy
Processing file: /home/focus/Bureau/Audio_Node/src/resources/ray_audio/features/audioset_v1_embeddings/bal_train/d3.tfrecord
Processing file: /home/focus/Bureau/Audio_Node/src/resources/ray_audio/features/audioset_v1_embeddings/bal_train/Dh.tfrecord
Processing file: /home/focus/Bureau/Audio_Node/src/resources/ray_audio/features/audioset_v1_embeddings/bal_train/77.tfrecord
Processing file: /home/focus/Bureau/Audio_Node/src/resources/ray_audio/features/audioset_v1_embeddings/bal_train/01.tfrecord
Processing file: /home/focus/Burea